In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyvallocation.views import BlackLittermanProcessor
from pyvallocation.portfolioapi import AssetsDistribution, PortfolioWrapper
from pyvallocation.utils.projection import convert_scenarios_compound_to_simple, project_mean_covariance, log2simple
from pyvallocation import probabilities

# load daily close data for some ETFs
df = pd.read_csv('../examples/ETF_prices.csv',index_col=0,parse_dates=True)
print('Loaded ', df.columns)

# resample to weekly frequency
weekly_prices = df.resample('W').ffill()

# compute compounded returns
weekly_returns = np.log(weekly_prices).diff().dropna()

# store the returns shape
T, N = weekly_returns.shape

# inputs/parameters
ANNUALIZATION_FACTOR = 52
HALF_LIFE = ANNUALIZATION_FACTOR * 10 # 10 years
TARGET_RETURN = 0.06/ANNUALIZATION_FACTOR

INVESTMENT_HORIZON = 1 # weeks ahead
TARGET_RETURN_HORIZON = TARGET_RETURN*INVESTMENT_HORIZON

Loaded  Index(['DBC', 'GLD', 'SPY', 'TLT'], dtype='object')


In [2]:
from pyvallocation import moments, probabilities
p = probabilities.generate_uniform_probabilities(T)
mu, cov = moments.estimate_sample_moments(weekly_returns,p)

mu_ = moments.shrink_mean_jorion(mu,cov,T)
cov_ = moments.shrink_covariance_ledoit_wolf(weekly_returns,cov,target='identity')

mu_hor, cov_hor = project_mean_covariance(mu_,cov_,INVESTMENT_HORIZON)

mu_hor_simple, cov_hor_simple = log2simple(mu_hor,cov_hor)

PortfolioWrapper(AssetsDistribution(mu=mu_hor_simple,cov=cov_hor_simple)).get_portfolios_return_constraint(TARGET_RETURN_HORIZON)[0].round(3)

Setting default optimizer, call initialize_optimizer to customize it
No custom constraints specified, using default ones


DBC    0.168
GLD    0.104
SPY    0.237
TLT    0.490
dtype: float64

In [3]:
mu_bl, cov_bl = BlackLittermanProcessor(
    prior_mean=mu_,
    prior_cov=cov_,
    mean_views={'SPY':-0.05/ANNUALIZATION_FACTOR},
    view_confidences=1,
    tau=1,
    idzorek_use_tau=True,
    omega='idzorek',
    verbose=True,
).get_posterior()

print(mu*ANNUALIZATION_FACTOR)
print(mu_bl*ANNUALIZATION_FACTOR)
mu_bl_hor,cov_bl_hor = log2simple(*project_mean_covariance(mu_bl,cov_bl,INVESTMENT_HORIZON))

PortfolioWrapper(AssetsDistribution(mu=mu_bl_hor,cov=cov_bl_hor)).get_portfolios_return_constraint(TARGET_RETURN_HORIZON)[0].round(3)

Setting default optimizer, call initialize_optimizer to customize it
No custom constraints specified, using default ones


[BL] π source: prior_mean.
[BL] Built P (1, 4), Q (1, 1).
[BL] Ω from Idzorek confidences (base = τ Σ).
[BL] Posterior mean and covariance computed.
DBC    0.006164
GLD    0.089540
SPY    0.097284
TLT    0.027575
Name: mu, dtype: float64
DBC   -0.033582
GLD    0.063791
SPY   -0.050000
TLT    0.060013
dtype: float64


DBC    0.084
GLD    0.249
SPY    0.176
TLT    0.491
dtype: float64

In [4]:
pw = PortfolioWrapper(AssetsDistribution(mu=mu_bl_hor,cov=cov_bl_hor))
pw.initialize_optimizer('CVaR',tcost_lambda=1,prev_weights=np.array([0,0,1,0]))
pw.get_portfolios_return_constraint(TARGET_RETURN_HORIZON)[0].round(3)

No custom constraints specified, using default ones
Computing scenarios from mu and cov, using distribution: Normal


DBC    0.000
GLD    0.420
SPY    0.256
TLT    0.324
dtype: float64